<a href="https://colab.research.google.com/github/rachit2005/Large-Language-Model/blob/main/makemore_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#We are using MLP

In [ ]:
import requests

url = "https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt"
resp = requests.get(url)

words = resp.text.splitlines()
print(words[:3])

['emma', 'olivia', 'ava']


In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
chars  = sorted(list(set(''.join(words))))

# building a vocabulary
stoi = {c:i+1 for i,c in enumerate(chars)}
stoi['.'] = 0
itos = {value:key for key , value in stoi.items()}

print(f"sting to index: {stoi}")
print(f"index to string: {itos}")
len(words)

sting to index: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
index to string: {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


32033

In [ ]:
block_size = 3 # --> context lenght : how many charaters do we take to predict the next one ?
X , Y = [] , []

print("X in tensor format --> Y in tensor format")
for word in words:
  print(word)

  context = [0]*block_size
  for ch in word + '.':
    # '.' shows the word is ended
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)

    # print(''.join(itos[i] for i in context) , '--> ' , itos[ix])

    context = context[1:] + [ix] # crop and append

X = torch.tensor(X) # first thre chars of the word
Y = torch.tensor(Y) # 4th char of the word to predict

# print(X,Y)

Streaming output truncated to the last 5000 lines.
doron
dovi
draysen
dredyn
dugan
durell
ebin
edwardo
efrem
ehaan
ehab
eirik
eldar
eliav
eligio
eloi
eluzer
elway
emaad
emeka
enki
ero
erron
esaiah
estiven
eulalio
everet
everly
evert
evian
eyas
ezer
ezren
faaris
facundo
famous
farzan
felton
field
filipe
finbar
finch
flynt
fordham
fouad
gabryel
garnet
gaurik
gavi
geovonni
germany
gerrard
gevorg
giuliani
graden
gradyn
grayer
grayton
greylen
grimm
gurshaan
hameed
hanzel
hasib
haylen
henning
herminio
hill
hiyan
hrihaan
huxson
hykeem
ihsaan
imer
imir
iremide
isahi
isaih
ishak
ishank
issachar
issam
issiah
iyaan
izai
izick
jabbar
jabri
jaccob
jadarrius
jadin
jaecion
jaelin
jaelon
jaevon
jah
jahcere
jahmeer
jahvon
jahzir
jaidin
jaishawn
jakaree
jakorey
jalan
jameis
jansiel
janthony
janziel
jarom
jaseer
jaston
jatavion
jathniel
javarius
jawuan
jaxel
jaxxyn
jaxzon
jayace
jaydien
jayke
jaymere
jayniel
jayonni
jayston
jayziah
jazen
jazion
jeramie
jeramyah
jerek
jeremey
jeren
jeric
jerman
jermani
je

In [ ]:
X.shape , Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [ ]:
# embeddings provide a richer, more informative way to represent categorical data like characters, allowing the model to capture semantic relationships and improve its performance.

C = torch.randn((27,2)) # each 27 characters will have 2Dim embedding

In [ ]:
# C[X] is looking up the embedding for each character index present in the X tensor

# The emb tensor has a shape of (number of training examples, block_size, embedding dimension).
emb = C[X]
emb , emb.shape

(tensor([[[ 0.8255, -0.3132],
          [ 0.8255, -0.3132],
          [ 0.8255, -0.3132]],
 
         [[ 0.8255, -0.3132],
          [ 0.8255, -0.3132],
          [ 0.5716, -0.1275]],
 
         [[ 0.8255, -0.3132],
          [ 0.5716, -0.1275],
          [ 0.4381,  0.2583]],
 
         ...,
 
         [[-0.0888, -0.1605],
          [-0.0888, -0.1605],
          [ 0.9762,  0.0715]],
 
         [[-0.0888, -0.1605],
          [ 0.9762,  0.0715],
          [-0.0888, -0.1605]],
 
         [[ 0.9762,  0.0715],
          [-0.0888, -0.1605],
          [ 0.9228,  1.7292]]]),
 torch.Size([228146, 3, 2]))

In [ ]:
# emb[:, 0, :] selects all examples (:) and the first position within the block (0), taking all dimensions of the embedding (:). This effectively extracts the embeddings for the first character in each context block.
# emb[:, 1, :] selects all examples (:) and the second position within the block (1), taking all dimensions of the embedding (:). This extracts the embeddings for the second character in each context block.
# emb[:, 2, :] selects all examples (:) and the third position within the block (2), taking all dimensions of the embedding (:). This extracts the embeddings for the third character in each context block.
emb[:,0,:] , emb[:,1,:] , emb[:,2,:]

torch.unbind(emb)

(tensor([[ 0.8255, -0.3132],
         [ 0.8255, -0.3132],
         [ 0.8255, -0.3132]]),
 tensor([[ 0.8255, -0.3132],
         [ 0.8255, -0.3132],
         [ 0.5716, -0.1275]]),
 tensor([[ 0.8255, -0.3132],
         [ 0.5716, -0.1275],
         [ 0.4381,  0.2583]]),
 tensor([[ 0.5716, -0.1275],
         [ 0.4381,  0.2583],
         [ 0.4381,  0.2583]]),
 tensor([[ 0.4381,  0.2583],
         [ 0.4381,  0.2583],
         [-0.1410,  0.2691]]),
 tensor([[ 0.8255, -0.3132],
         [ 0.8255, -0.3132],
         [ 0.8255, -0.3132]]),
 tensor([[ 0.8255, -0.3132],
         [ 0.8255, -0.3132],
         [ 1.3090,  0.4037]]),
 tensor([[ 0.8255, -0.3132],
         [ 1.3090,  0.4037],
         [ 0.2281,  0.5031]]),
 tensor([[ 1.3090,  0.4037],
         [ 0.2281,  0.5031],
         [ 1.1352, -1.2006]]),
 tensor([[ 0.2281,  0.5031],
         [ 1.1352, -1.2006],
         [ 0.9708,  1.9938]]),
 tensor([[ 1.1352, -1.2006],
         [ 0.9708,  1.9938],
         [ 1.1352, -1.2006]]),
 tensor([[ 0.9708,  1

In [ ]:
# torch.cat(torch.unbind(emb , dim=1) , dim=1) # --> embedding of 1st , 2nd , 3rd character of X in 1 row

In [ ]:
# emb.view([emb.shape[0],6]) == torch.cat(torch.unbind(emb , dim=1) , dim=1)

In [ ]:
# --------------------- Properly aranged -------------------------------------

In [ ]:
# dataset

X.shape , Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [ ]:
def build_dataset(words):
  block_size = 3 # --> context lenght : how many charaters do we take to predict the next one ?
  X , Y = [] , []

  # print("X in tensor format --> Y in tensor format")
  for word in words:
    # print(word)

    context = [0]*block_size
    for ch in word + '.':
      # '.' shows the word is ended
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)

      # print(''.join(itos[i] for i in context) , '--> ' , itos[ix])

      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X) # first thre chars of the word
  Y = torch.tensor(Y) # 4th char of the word to predict
  print(X.shape , Y.shape)
  return X , Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr , Ytr = build_dataset(words[:n1])
Xdev , Ydev = build_dataset(words[n1:n2])
Xtest , Ytest = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,10),generator=g) # -->This is done to improve model efficiency, generalization, and the ability to capture meaningful relationships in the data


'''using kaiming initialisation '''
w1 = torch.randn((30,400),generator=g) * (5/3)/((30**0.5))
b1 = torch.randn(400,generator=g)

w2 = torch.randn((400,27),generator=g) * 0.1
b2 = torch.randn(27,generator=g)*0

parameters = [C,w1,b1,w2,b2]

for p in parameters:
  p.requires_grad = True

In [ ]:
losses = []

for i in range(10000):

  # minibatch construc
  ix = torch.randint(0,Xtr.shape[0],(32,))
  X_batch = Xtr[ix]
  Y_batch = Ytr[ix]

  # forward pass
  emb = C[X_batch] # [32 , 3 , 10]
  flatten_layer = emb.view(-1,30) # [32, 30]
  first_layer = flatten_layer @ w1 + b1 # [32 , 100]
  # first_layer.shape
  tanh_layer = torch.tanh(first_layer)
  logits = tanh_layer @ w2 + b2 # [32 , 27]

  loss = F.cross_entropy(logits,Y_batch)

  for p in parameters:
    p.grad = None

  loss.backward()

  # gradient descent
  lr = 0.1 if i < 10000 else 0.01 # learning rate annealing
  for p in parameters:
    if p.grad is not None:
        p.data += -lr * p.grad

print(loss.item())


2.1484673023223877


In [ ]:
# dev loop
emb = C[Xdev] # [137 , 3 ,2]
first_layer = emb.view(-1 , 30) @w1 + b1 # [137 , 100]
tanh_layer = torch.tanh(first_layer)
logits = tanh_layer @w2 + b2 # [137 , 27]

loss = F.cross_entropy(logits,Ydev)
loss.item()

2.2256217002868652

In [ ]:
# test loop
emb = C[Xtest] # [137 , 3 ,2]
first_layer = emb.view(-1 , 30) @w1 + b1 # [137 , 100]
tanh_layer = torch.tanh(first_layer)
logits = tanh_layer @w2 + b2 # [137 , 27]

loss = F.cross_entropy(logits,Ytest)
loss.item()

2.227261543273926

In [ ]:
# sampling from the model

g = torch.Generator().manual_seed(2147483647 + 18)

for _ in range(10):
  out = []
  context = [0]*block_size

  while True:
    emb = C[torch.tensor(context)] # [block_size , embbed_dim]
    flatten_layer = emb.view(-1,30) # [1 , 30]
    first_layer = torch.tanh(flatten_layer @ w1 + b1) # [1 , 400]
    logits = first_layer @ w2 + b2 # [1 , 27]
    probs = F.softmax(logits , dim=1) # [1 , 27]

    ix = torch.multinomial(probs , num_samples=1 , replacement=True , generator=g).item()

    context = context[1:] + [ix]

    out.append(ix)

    if ix == 0:
      break

  print(''.join(itos[i] for i in out))


jase.
cariance.
erelin.
eno.
kinilynno.
jely.
allien.
jehiton.
tama.
qufbyryaregh.
